<a href="https://colab.research.google.com/github/HuaTrung/Colab/blob/master/Logictis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression 
import seaborn as sns
import matplotlib as plt

In [0]:
class MyLinearRegression:
    def __init__(self,x,y,degree):
        self.size=len(x)
        self.x_train_plot = x
        self.x_train=self.map_feature(degree)
        self.y_train = y.reshape(-1,1)
        self.theta=np.zeros(x.shape[1]+1)
    
    def costFunction():
      return np.sum(1/(2*size)*(x.dot(theta)-y.flatten())**2)
    
    def gradientDescent(self):
        lerning_rate=0.01
        self.theta=self.theta-np.sum((np.sum(self.x_train*self.theta,axis=1).reshape(-1,1)-self.y_train)*self.x_train,axis=0)*lerning_rate/self.size
        return self.theta
        
    def plotConverage(self):
        itter=[]
        costfunction=[]
        for i in range(0,100):
            self.theta=self.gradientDescent()
            itter.append(i)
            costfunction.append(self.costFunction())
        plt.ylabel('Cost Function')
        plt.xlabel('Iterater')
        plt.plot(np.array(itter), np.array(costfunction),'-')
        plt.rcParams["figure.figsize"] = [4,3]
        plt.show()
        
    def trainHard(self):
        for i in range(0,5000):
            self.theta=self.gradientDescent()
            
    def plotLine(self):
        self.y_result=np.sum(self.theta*self.x_train,axis=1)
        plt.plot(self.x_train_plot, self.y_train, 'ro',markersize=3)
        plt.plot(self.x_train_plot, self.y_result, '-')
        plt.xlabel('X ASIX')
        plt.ylabel('Y ASIX')
        plt.rcParams["figure.figsize"] = [4,3]
        plt.show()

    def CompareAccuracy(self):
        clf = LinearRegression(normalize=True)
        clf.fit(self.x_train_plot,self.y_train)
        sk_y_pred = clf.predict(self.x_train_plot)
        me_y_pred=np.sum(self.theta*self.x_train,axis=1)
        print ("sklearn: ")
        print(r2_score(self.y_train,sk_y_pred))
        print ("Me: ")
        print(r2_score(self.y_train,me_y_pred))

    def standardize(self,data):
        return (data - np.mean(data))/(np.max(data) - np.min(data))
    
    def map_feature(self,degree):
        x=self.x_train_plot
        pram=[]
        for i in range(x.shape[1]):
            pram.append([x[:,i],i,i+1])
        result=np.array([x[:,i] for i in range(x.shape[1])]).T
        listCurPar=result
        result=self.standardize(result)
        for i in range(2,degree+1):
            list_temp=[]

            for index in range(len(pram)):  
                temp=0;
                for j in range(pram[index][1],listCurPar.shape[1]):
                    temp=temp+1
                    list_temp.append((pram[index][0]*listCurPar[:,j]))

                if(pram[index][1]==0):
                    pram[index][1]=0
                    pram[index][2]=pram[index][1]+temp
                else:
                    pram[index][1]=pram[index-1][2]
                    pram[index][2]=pram[index][1]+temp
            listCurPar=np.array(list_temp).T;
            for i in list_temp:
                result=np.append(result,self.standardize(i.reshape(-1,1)),axis=1)
        self.x_train=np.append(np.ones(x.shape[0]).reshape(-1,1),result,axis=1)
        self.theta=np.zeros(self.x_train.shape[1])
    
    def poly(sefl):
        x=self.x_train
        pram=[]
        for i in range(x.shape[1]):
            pram.append([x[:,i],i,i+1])
        result=np.array([x[:,i] for i in range(x.shape[1])]).T
        listCurPar=result
        for i in range(2,degree):
            list_temp=[]

            for index in range(len(pram)):  
                temp=0;
                for j in range(pram[index][1],listCurPar.shape[1]):
                    temp=temp+1
                    list_temp.append((pram[index][0]*listCurPar[:,j]))

                if(pram[index][1]==0):
                    pram[index][1]=0
                    pram[index][2]=pram[index][1]+temp
                else:
                    pram[index][1]=pram[index-1][2]
                    pram[index][2]=pram[index][1]+temp
            listCurPar=np.array(list_temp).T;
            for i in list_temp:
                result=np.append(result,i.reshape(-1,1),axis=1)
        self.x_train=np.append(np.ones(x.shape[0]).reshape(-1,1),result,axis=1)
        

In [0]:
def generatePolyPoints(start, stop, num_points, coeff, noiseLevel = 1, plot = 1):
    """
    Generate evenly spaced x and y coordinates in Euclidean space from a polynomial
    with gaussian noise    
    
    Parameters
    ------------
    start : float, 
          Minimum value for the generated x-coordinates
    
    stop  : float
          Maximum value for the generated x-coordinates
    
    num_points : int
               Number of coordinates to generate
    
    coeff : 1-d array, 
    
    noiseLevel : float, optional
              Indicates amount of gaussian noise to add to the y-coordinates
              generated from the polynomial characterized by coeff array. 
              Zero indicates no noise, while each integer increment results in
              ten-fold noise increase
    
    plot : int, values = 1|~1 , optional
        plot == 1 returns a matplotlib plot of the generated coordinates
    
    Returns
    -----------       
    x_pts : 1-d array, shape = [num_points,]     
    y_pts : 1-d array, shape = [num_points,]
    """
    x_pts = np.arange(start, stop, (stop - start)/num_points)
    line = coeff[0]
    
    for i in np.arange(1, len(coeff)):          
        line += coeff[i] * x_pts ** i 

    if noiseLevel > 0:
        y_pts = np.random.normal(-(10 ** noiseLevel), 10 ** noiseLevel, len(x_pts)) + line
    else:
        y_pts = line

    if plot == 1: #Plot option
        plt.figure()
        plt.scatter(x_pts, y_pts)
        plt.xlabel('x')
        plt.ylabel('y')
        
    return x_pts, y_pts

In [0]:
x_pts, y_pts = generatePolyPoints(0, 50, 100, [5, 1, 1], 
                                  noiseLevel = 2, plot = 0)

In [0]:
test2= MyLinearRegression(x_pts.reshape(-1,1),y_pts.reshape(-1,1))

In [0]:
test2.map_feature(1)